In [1]:
import pandas as pd

In [2]:
bizdf = pd.read_json("yelp_academic_dataset_business.json", lines=True)
bizdf.dropna(inplace=True)

bizdf.drop(['hours','is_open','review_count'], axis=1, inplace=True)


bizdf = bizdf[bizdf.categories.str.contains('Restaurant')]
print(len(bizdf))



size = 1000000
review = pd.read_json('yelp_academic_dataset_review.json', lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

42674


In [3]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(bizdf, chunk_review, on='business_id', how='inner')
    
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    
    chunk_list.append(chunk_merged)

    # After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

625002 out of 1,000,000 related reviews
618428 out of 1,000,000 related reviews
625784 out of 1,000,000 related reviews
617118 out of 1,000,000 related reviews
603639 out of 1,000,000 related reviews
617238 out of 1,000,000 related reviews
621583 out of 1,000,000 related reviews
624598 out of 1,000,000 related reviews
390568 out of 1,000,000 related reviews


In [2]:
df = pd.read_csv("scraped_output.csv")

In [3]:
df.dropna(inplace=True)
df.head(3)

,Unnamed: 0,Review,Date,Stars,Restaurant
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB


In [4]:
import spacy
import deplacy
from spacytextblob.spacytextblob import SpacyTextBlob
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')


In [12]:
### REFERENCE FROM SUKRUT
text = "The food and ambience was good and bad respectively"

doc = en(text)
deplacy.render(doc)

seen = set() # keep track of covered words

chunks = []
for sent in doc.sents:
    heads = [cc for cc in sent if cc.dep_ == 'conj']
    for head in heads:
        words = [None]*100
        counter = 0
        for i in head.subtree:
            if(i in heads):
                if(i != head):
                    break
            words[counter] = i
            counter = counter + 1
            if(i.dep_ == 'cc'):# and if i.pos_ == punct
                break
        res = []
        for val in words:
            if val != None :
                res.append(val)
        for word in res:
            seen.add(word)
        chunk = (' '.join([ww.text for ww in res]))
        chunks.append( (head.i, chunk) )
    unseen = [ww for ww in sent if ww not in seen]
    chunk = ' '.join([ww.text for ww in unseen if ww.dep_ != 'cc'])
    chunks.append( (sent.root.i, chunk) )

chunks = sorted(chunks, key=lambda x: x[0])

for ii, chunk in chunks:
    print("The chunk is: " + chunk)
    doc = en(chunk)
    for i in doc.sents:
        for cc in i:
            if cc.dep_ == 'nsubj':
                print("Noun :" + cc.text)
    print("Polarity: " + str(doc._.polarity))

The          DET   <════╗     det
food         NOUN  ═╗═╗═╝<══╗ nsubj
and          CCONJ <╝ ║     ║ cc
ambience     NOUN  <══╝     ║ conj
was          AUX   ═════╗═╗═╝ ROOT
good         ADJ   ═╗═╗<╝ ║   acomp
and          CCONJ <╝ ║   ║   cc
bad          ADJ   <══╝   ║   conj
respectively ADV   <══════╝   advmod
The chunk is: ambience
Polarity: 0.0
The chunk is: The food was good respectively
Noun :food
Polarity: 0.35
The chunk is: bad
Polarity: -0.6999999999999998


In [5]:
# Function to do feature-descriptor extraction from each row:
def get_feature_descriptors(text):
#     text = row["text"]

    doc = en(text)
#     deplacy.render(doc)
    seen = set() # keep track of covered words

    chunks = []
    for sent in doc.sents:
        heads = [cc for cc in sent if cc.dep_ == 'conj']
        for head in heads:
            words = [None]*100
            counter = 0
            for i in head.subtree:
                if(i in heads):
                    if(i != head):
                        break
                words[counter] = i
                counter = counter + 1
                if(i.dep_ == 'cc'):
                    break
            res = []
            for val in words:
                if val != None :
                    res.append(val)
            for word in res:
                seen.add(word)
            chunk = (' '.join([ww.text for ww in res]))
            chunks.append( (head.i, chunk) )
        unseen = [ww for ww in sent if ww not in seen]
        chunk = ' '.join([ww.text for ww in unseen if ww.dep_ != 'cc'])
        chunks.append( (sent.root.i, chunk) )

    chunks = sorted(chunks, key=lambda x: x[0])
    retdict = dict()
    for ii, chunk in chunks:
#         print("The chunk is: " + chunk)
        doc = en(chunk)
        for i in doc.sents:
            for cc in i:
                if cc.pos_ == 'NOUN':
                    print("Noun :" + cc.text)
                    if cc.text in retdict:
                        nlist = retdict[cc.text]
                        nlist.append(chunk)
                        retdict[cc.text] = nlist;
                    else:
                        retdict[cc.text] = [chunk]
                        
#         print("Polarity: " + str(doc._.polarity))
    return retdict

In [6]:
# trialdf = df.copy()
trialdf = df#.sample(100, random_state=4222)
# trialdf

trialdf["feature_descriptors"] = trialdf["Review"].apply(lambda x:get_feature_descriptors(x))

# for i in tqdm(range(0, len(df))):
    

Noun :hamburgers
Noun :cocktails
Noun :night
Noun :room
Noun :temperature
Noun :degrees
Noun :burgers
Noun :bread
Noun :meat
Noun :time
Noun :pm
Noun :date
Noun :wife
Noun :way
Noun :errands
Noun :Wife
Noun :no.1
Noun :touch
Noun :servicing
Noun :sizes
Noun :size
Noun :discerning
Noun :burgers
Noun :taste
Noun :ingredients
Noun :price
Noun :sides
Noun :onion
Noun :rings
Noun :dope
Noun :top
Noun :service
Noun :burgers
Noun :thing
Noun :weather
Noun :onion
Noun :rings
Noun :char
Noun :discount
Noun :flavors
Noun :salty
Noun :way
Noun :crispy
Noun :crust
Noun :discount
Noun :price
Noun :place
Noun :burgers
Noun :number
Noun :number
Noun :Meat
Noun :cheese
Noun :sauce
Noun :veggies
Noun :burger
Noun :burgers
Noun :milkshakes
Noun :fries
Noun :lunch
Noun :promo
Noun :way
Noun :deal
Noun :vouchers
Noun :Burgers
Noun :patty
Noun :brioches
Noun :buns
Noun :wine
Noun :bottles
Noun :degrees
Noun :breakfast
Noun :eggs
Noun :sausage
Noun :way
Noun :Hash
Noun :brown
Noun :crispy
Noun :potato
Noun 

Noun :burgers
Noun :dipping
Noun :sauce
Noun :Food
Noun :number
Noun :burgers
Noun :Thanks
Noun :aldrick
Noun :service
Noun :night
Noun :queue
Noun :mins
Noun :table
Noun :burger
Noun :price
Noun :quality
Noun :number
Noun :burger
Noun :fries
Noun :queue
Noun :evening
Noun :service
Noun :burger
Noun :pm
Noun :night
Noun :meal
Noun :beef
Noun :place
Noun :Degrees
Noun :outlet
Noun :time
Noun :burgers
Noun :one
Noun :level
Noun :No.1
Noun :flavours
Noun :milder
Noun :flavours
Noun :number
Noun :burgers
Noun :menu
Noun :burgers
Noun :meat
Noun :buns
Noun :fries
Noun :Service
Noun :ambience
Noun :road
Noun :noise
Noun :restaurant
Noun :setting
Noun :milkshake
Noun :milk
Noun :potato
Noun :chips
Noun :cream
Noun :milkshake
Noun :cookies
Noun :apple
Noun :tart
Noun :place
Noun :drinks
Noun :food
Noun :hours
Noun :burgs
Noun :milkshake
Noun :️
Noun :number
Noun :order
Noun :onion
Noun :rings
Noun :burgers
Noun :vanilla
Noun :milkshake
Noun :burrata
Noun :one
Noun :number
Noun :pesto
Noun :pep

Noun :Staff
Noun :training
Noun :guest
Noun :mins
Noun :friend
Noun :pm
Noun :visit
Noun :outlet
Noun :number
Noun :burger
Noun :crescenza
Noun :cheese
Noun :strips
Noun :bacon
Noun :bun
Noun :dinner
Noun :set
Noun :carte
Noun :burger
Noun :fries
Noun :fries
Noun :meal
Noun :price
Noun :burgers
Noun :price
Noun :hour
Noun :promotion
Noun :beer
Noun :tap
Noun :sides
Noun :lobster
Noun :bun
Noun :😋
Noun :seats
Noun :place
Noun :burgers
Noun :lunchtime
Noun :cheeseburger
Noun :size
Noun :greasy
Noun :fries
Noun :people
Noun :hand
Noun :waitress
Noun :burgers
Noun :menu
Noun :place
Noun :connoisseur
Noun :experts
Noun :questions
Noun :bucks
Noun :taxi
Noun :fares
Noun :place
Noun :hype
Noun :talk
Noun :foward
Noun :burgers
Noun :Restaurant
Noun :ambience
Noun :lots
Noun :choices
Noun :drinks
Noun :food
Noun :burger
Noun :burgers
Noun :burgers
Noun :price
Noun :point
Noun :shakes
Noun :burger
Noun :milkshake
Noun :Fries
Noun :winners
Noun :kick
Noun :spice
Noun :end
Noun :wings
Noun :fries


Noun :service
Noun :price
Noun :food
Noun :quality
Noun :view
Noun :street
Noun :rock
Noun :burger
Noun :house
Noun :onion
Noun :rings
Noun :budget
Noun :occasions
Noun :prices
Noun :medium
Noun :bun
Noun :bit
Noun :Burgers
Noun :onion
Noun :rings
Noun :burgers
Noun :place
Noun :burger
Noun :wine
Noun :menu
Noun :place
Noun :channel
Noun :star
Noun :Customer
Noun :service
Noun :lettuce
Noun :tomato
Noun :onion
Noun :burger
Noun :Service
Noun :expectations
Noun :place
Noun :people
Noun :patties
Noun :food
Noun :chillout
Noun :place
Noun :service
Noun :selection
Noun :beers
Noun :food
Noun :burgers
Noun :price
Noun :SGD$30
Noun :reviews
Noun :hamburgers
Noun :hamburger
Noun :meat
Noun :Fries
Noun :onion
Noun :rings
Noun :Staff
Noun :customer
Noun :service
Noun :food
Noun :places
Noun :milkshakes
Noun :sides
Noun :Burgers
Noun :burgers
Noun :burger
Noun :fries
Noun :burgers
Noun :waiters
Noun :experience
Noun :service
Noun :place
Noun :hour
Noun :pm
Noun :places
Noun :order
Noun :hour
Nou

Noun :restaurant
Noun :thrice
Noun :Food
Noun :point
Noun :lobster
Noun :dishes
Noun :go
Noun :to
Noun :Fish
Noun :portion
Noun :fish
Noun :potato
Noun :Chips
Noun :rib
Noun :eye
Noun :par
Noun :steak
Noun :service
Noun :food
Noun :atmosphere
Noun :birthday
Noun :staff
Noun :arrangements
Noun :place
Noun :Steak
Noun :par
Noun :restaurants
Noun :soup
Noun :point
Noun :chill
Noun :vibes
Noun :place
Noun :oysters
Noun :mussels
Noun :twister
Noun :fries
Noun :greasy
Noun :taste
Noun :food
Noun :service
Noun :server
Noun :manager
Noun :service
Noun :foods
Noun :service
Noun :steak
Noun :place
Noun :office
Noun :time
Noun :night
Noun :dinner
Noun :service
Noun :care
Noun :dinner
Noun :service
Noun :Food
Noun :burgers
Noun :ambience
Noun :place
Noun :space
Noun :tables
Noun :conversation
Noun :party
Noun :pity
Noun :menu
Noun :items
Noun :specials
Noun :stars
Noun :ambience
Noun :moment
Noun :restaurant
Noun :stars
Noun :service
Noun :table
Noun :foods
Noun :service
Noun :restaurant
Noun :Wif

...

Noun :step
Noun :crab
Noun :cake
Noun :starter
Noun :rib
Noun :eye
Noun :style
Noun :bone
Noun :fan
Noun :food
Noun :time
Noun :food
Noun :friend
Noun :staff
Noun :atmosphere
Noun :circuit
Noun :breaker
Noun :Apple
Noun :menu
Noun :vegetables
Noun :sprouts
Noun :pumpkin
Noun :style
Noun :anniversary
Noun :meal
Noun :wld
Noun :sofa
Noun :seats
Noun :occasion
Noun :Boyfriend
Noun :anniversary
Noun :bf
Noun :eye
Noun :steak
Noun :sirloin
Noun :steaks
Noun :texture
Noun :lobster
Noun :dinner
Noun :time
Noun :wife
Noun :birthday
Noun :course
Noun :service
Noun :night
Noun :Course
Noun :+
Noun :Side
Noun :level
Noun :service
Noun :decadence
Noun :meal
Noun :dish
Noun :steaks
Noun :perfection
Noun :food
Noun :wine
Noun :service
Noun :Steaks
Noun :char
Noun :perfection
Noun :Apple
Noun :crumble
Noun :course
Noun :tasting
Noun :menu
Noun :steak
Noun :shoutout
Noun :dinner
Noun :experience
Noun :initiative
Noun :work
Noun :sight
Noun :whisky
Noun :steak
Noun :Food
Noun :baby
Noun :chicken
Noun :

Noun :dinner
Noun :birthday
Noun :celebration
Noun :level
Noun :service
Noun :attention
Noun :detail
Noun :staff
Noun :time
Noun :card
Noun :wishes
Noun :steak
Noun :steak
Noun :wife
Noun :steak
Noun :sauce
Noun :name
Noun :steak
Noun :salt
Noun :pepper
Noun :staffs
Noun :corn
Noun :sides
Noun :additions
Noun :service
Noun :flavoring
Noun :service
Noun :food
Noun :money
Noun :dinner
Noun :Perfection
Noun :Attention
Noun :detail
Noun :food
Noun :staff
Noun :time
Noun :service
Noun :host
Noun :bartender
Noun :level
Noun :service
Noun :details
Noun :needs
Noun :attention
Noun :anniversary
Noun :staff
Noun :time
Noun :anniversary
Noun :card
Noun :desert
Noun :food
Noun :experience
Noun :wife
Noun :anniversary
Noun :Steak
Noun :types
Noun :steaks
Noun :cuts
Noun :pepper
Noun :sauce
Noun :charge
Noun :sauce
Noun :meat
Noun :ambience
Noun :feeling
Noun :bone
Noun :marrow
Noun :flan
Noun :flavour
Noun :seabass
Noun :kaya
Noun :pricy
Noun :food
Noun :star
Noun :Service
Noun :premium
Noun :mood


Noun :ambiance
Noun :steak
Noun :lobster
Noun :grass
Noun :top
Noun :caviar
Noun :dinner
Noun :steaks
Noun :Staff
Noun :ambiance
Noun :restaurant
Noun :wolves
Noun :price
Noun :tag
Noun :cut
Noun :meat
Noun :steak
Noun :study
Noun :degree
Noun :taste
Noun :ribeye
Noun :difference
Noun :strip
Noun :price
Noun :tag
Noun :latter
Noun :moisture
Noun :place
Noun :atmosphere
Noun :staff
Noun :place
Noun :steak
Noun :star
Noun :restaurant
Noun :end
Noun :steak
Noun :steak
Noun :wine
Noun :steakhouse
Noun :prices
Noun :fillet
Noun :place
Noun :ditto
Noun :staff
Noun :steak
Noun :snow
Noun :wagyu
Noun :sushi
Noun :Beef
Noun :Beef
Noun :mouth
Noun :Beef
Noun :slider
Noun :beef
Noun :burget
Noun :ground
Noun :beef
Noun :beef
Noun :steak
Noun :diner
Noun :experience
Noun :restaurant
Noun :design
Noun :music
Noun :food
Noun :experience
Noun :service
Noun :staff
Noun :day
Noun :book
Noun :table
Noun :procedure
Noun :member
Noun :point
Noun :deduction
Noun :bill
Noun :request
Noun :bill
Noun :settlem

Noun :staff
Noun :dish
Noun :setting
Noun :choice
Noun :deserts
Noun :wife
Noun :location
Noun :shops
Noun :drinks
Noun :show
Noun :drinks
Noun :service
Noun :place
Noun :gossip
Noun :people
Noun :dining
Noun :cocktails
Noun :service
Noun :food
Noun :honeymoon
Noun :wine
Noun :service
Noun :steak
Noun :location
Noun :environment
Noun :dinner
Noun :course
Noun :menu
Noun :evening
Noun :dining
Noun :70++
Noun :price
Noun :food
Noun :steak
Noun :sides
Noun :effort
Noun :steak
Noun :Service
Noun :Wine
Noun :selection
Noun :bar
Noun :food
Noun :service
Noun :bartenders
Noun :steaks
Noun :steak
Noun :chicken
Noun :chicken
Noun :shelf
Noun :martinis
Noun :music
Noun :playlist
Noun :husband
Noun :rib
Noun :eye
Noun :steak
Noun :steak
Noun :steak
Noun :service
Noun :ambiance
Noun :quality
Noun :steak
Noun :service
Noun :A5
Noun :cuts
Noun :menu
Noun :terms
Noun :taste
Noun :Celebrity
Noun :steakhouse
Noun :restaurant
Noun :quality
Noun :meats
Noun :steakhouses
Noun :%
Noun :price
Noun :expectat

Noun :setting
Noun :front
Noun :service
Noun :restaurant
Noun :boss
Noun :woman
Noun :place
Noun :Perfection
Noun :chair
Noun :dining
Noun :tables
Noun :glass
Noun :window
Noun :dining
Noun :chairs
Noun :tables
Noun :middle
Noun :restaurant
Noun :lounge
Noun :chairs
Noun :middle
Noun :server
Noun :ambience
Noun :Food
Noun :wine
Noun :stars
Noun :comments
Noun :chef
Noun :restaurant
Noun :waiter
Noun :food
Noun :bartender
Noun :mask
Noun :nose
Noun :period
Noun :dining
Noun :restaurant
Noun :view
Noun :casino
Noun :cuisine
Noun :Food
Noun :drinks
Noun :menu
Noun :hours
Noun :restaurant
Noun :menu
Noun :hour
Noun :prices
Noun :Service
Noun :meal
Noun :Art
Noun :sister
Noun :bar
Noun :floor
Noun :views
Noun :area
Noun :restaurant
Noun :meals
Noun :life
Noun :variety
Noun :flavours
Noun :ingredients
Noun :courses
Noun :course
Noun :odyssey
Noun :cooking
Noun :dishes
Noun :dinner
Noun :dish
Noun :experience
Noun :story
Noun :dinner
Noun :drinks
Noun :friends
Noun :service
Noun :team
Noun :b

In [24]:
trialdf.iloc[2, :]["feature_descriptors"]

{'burgers': ["One of the better burgers I 've had , the bread is soft , ."],
 'bread': ["One of the better burgers I 've had , the bread is soft , ."],
 'meat': ['the meat is thick and']}

In [7]:
trialdf

,Unnamed: 0,Review,Date,Stars,Restaurant,feature_descriptors
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB,"{'hamburgers': ['Good hamburgers , .'], 'cockt..."
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB,{'night': [' Place was extremely hot even tho...
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB,{'burgers': ['One of the better burgers I 've ...
5,5,"Diner time and date: 8 October 2021, 1pm. My w...",2 weeks ago,5 stars,TFDB,"{'time': ['Diner time : 8 October 2021 , 1 pm ..."
6,6,Excellent burgers with a taste of each ingredi...,2 weeks ago,5 stars,TFDB,{'burgers': ['Excellent burgers with a taste o...
...,...,...,...,...,...,...
4230,253,Nice little Italian place with warm and family...,5 years ago,5 stars,Art,{'place': ['Nice little Italian place with war...
4232,255,Classy Italian\n\nGreat Italian fare in a clas...,5 years ago,4 stars,Art,{'fare': ['Classy Italian Great Italian far...
4233,256,Best home made pasta ever,5 years ago,4 stars,Art,"{'home': ['Best home made pasta ever'], 'pasta..."
4235,258,Classic Italian restaurant in Raffles Place. M...,5 years ago,5 stars,Art,{'restaurant': ['Classic Italian restaurant in...


### Extract Features

In [19]:
import benepar, spacy
nlp = spacy.load('en_core_web_sm')
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
doc = nlp('The time for action is now. It is never too late to do something.')


[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [6]:
subclauses = []
for sent in list(doc.sents):
    print(sent._.parse_string)
    print(sent._.labels)
    print(list(sent._.children))
    subclauses.append(list(sent._.children))

subclauses = [item for sublist in subclauses for item in sublist]

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
('S',)
[The time for action, is now, .]
(S (NP (PRP It)) (VP (VBZ is) (ADVP (RB never)) (ADJP (RB too) (JJ late) (S (VP (TO to) (VP (VB do) (NP (NN something))))))) (. .))
('S',)
[It, is never too late to do something, .]


In [7]:
subclauses

[The time for action, is now, ., It, is never too late to do something, .]